In [0]:
import numpy as np

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
df=spark.table("Market").toPandas()
df=df.sort_values('Date').reset_index(drop=True)

df['Sensex_SMA_50']=df['Sensex_Close'].rolling(50).mean()
df['Sensex_SMA_200']=df['Sensex_Close'].rolling(200).mean()
df['Sensex_SMA_Signal']=0
df.loc[( (df['Sensex_SMA_50'] > df['Sensex_SMA_200']) & (df['Sensex_SMA_50'].shift(1) <= df['Sensex_SMA_200'].shift(1)) ), 'Sensex_SMA_Signal'] = 1
df['Sensex_SMA_Position']=np.where(df['Sensex_SMA_50']>df['Sensex_SMA_200'],1,0)
df['Sensex_Daily_Returns']=df['Sensex_Close'].pct_change()
df['Sensex_Cum_Daily_Returns']=(df['Sensex_Daily_Returns']+1).cumprod()

df['Sensex_SMA_Strategy']=df['Sensex_Daily_Returns']*df['Sensex_SMA_Position']
df['Sensex_Cum_SMA_Strategy']=(df['Sensex_SMA_Strategy']+1).cumprod()


In [0]:
df['Sensex_TR_Pct']=(df['Sensex_Close'].diff()/df['Sensex_Close'].shift(1)).abs()*100
df['Sensex_ATR_Pct']=df['Sensex_TR_Pct'].rolling(14).mean()
df['Sensex_ATR_Position']=0
threshold=df['Sensex_ATR_Pct'].median()
df.loc[(df['Sensex_SMA_50']>df['Sensex_SMA_200']) & (df['Sensex_TR_Pct']>threshold),'Sensex_ATR_Position']=1

In [0]:
df['Sensex_Middle_Band']=df['Sensex_Close'].rolling(20).mean()
df['Sensex_Lower_Band']=df['Sensex_Middle_Band']-(2*(df['Sensex_Middle_Band'].rolling(20).std()))

#Reversion Based (Ignoring Upper & Middle Band CrossOvers)
df['Sensex_BB_Signal_Rev']=0
df.loc[((df['Sensex_Close']<df['Sensex_Lower_Band'])&(df['Sensex_Close'].shift(1)>=df['Sensex_Lower_Band'].shift(1))),'Sensex_BB_Signal_Rev']=1
df['Sensex_BB_Position_Rev']=np.where(df['Sensex_Close']<df['Sensex_Lower_Band'],1,0)
df['Sensex_BB_Strategy_Rev']=df['Sensex_Daily_Returns']*df['Sensex_BB_Position_Rev']
df['Sensex_Cum_BB_Strategy_Rev']=(df['Sensex_BB_Strategy_Rev']+1).cumprod()

#Momentum Based (Ignoring Upper & Middle Band CrossOvers)
df['Sensex_BB_Signal']=0
df.loc[((df['Sensex_Close']>df['Sensex_Lower_Band'])&(df['Sensex_Close'].shift(1)<=df['Sensex_Lower_Band'].shift(1))),'Sensex_BB_Signal']=1
df['Sensex_BB_Strategy']=df['Sensex_Daily_Returns']*df['Sensex_BB_Signal']
df['Sensex_Cum_BB_Strategy']=(df['Sensex_BB_Strategy']+1).cumprod()

In [0]:
df['Sensex_ATR_BB_Position'] = 0
df.loc[(df['Sensex_Close'] > df['Sensex_Lower_Band']) & (df['Sensex_ATR_Pct'] > threshold), 'Sensex_ATR_BB_Position'] = 1

df['Sensex_ATR_BB_Strategy']=(df['Sensex_ATR_BB_Position']*df['Sensex_Daily_Returns'])
df['Sensex_Cum_ATR_BB_Strategy']=(df['Sensex_ATR_BB_Strategy']+1).cumprod()

In [0]:
from pyspark.sql import functions as F

strategy_sen=spark.createDataFrame(df)
strategy_sen=strategy_sen.withColumn('Year',F.year('Date')).withColumn('Month',F.month('Date'))
(
    strategy_sen.write.format("delta").mode("overwrite").option("overwriteSchema","true").partitionBy("year","month").saveAsTable("sen_strategy")
)

In [0]:
%sql
describe sen_strategy